# 场景布局生成

1. 根据文本提取陈设，并推导出布局，以及需要的资产。
3. 在资产库中选择符合要求的模型资产，并保存下资产的路径。
4. 结合上面的布局和资产将布局格式化为JSON格式文本，同时将解释翻译回中文
4. 将JSON文件转化为usda格式文本



### 推导布局

In [13]:
from langchain_community.chat_models import ChatOllama
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

llm = ChatOllama(model="qwen:32b-chat-v1.5-q5_K_M", temperature=0)

template = """阅读如下文本，回答问题。
文本：{context}
问题: 文本中包含哪些陈设？输出一个列表，尽可能详尽
"""

prompt = ChatPromptTemplate.from_template(template) | llm | StrOutputParser()

user_text = """
哈利波特推开门走进卧室，这是一间朴素的卧室，房间的布局简单而实用。一张单人床靠在左侧的墙上，上面铺着整齐的蓝色床单和柔软的羽绒被。
床边是一个简约的木质床头柜，上面放着一盏小巧的台灯和一本打开的魔法书。对面的墙上挂着一面镜子，下方是一张书桌，上面摆放着羽毛笔、墨水瓶和几本魔法课本。
书桌旁边是一个小巧的书架，书架上整齐地排列着各种魔法书籍和药草学资料。房间的一角，一个舒适的扶手椅静静地摆放着，旁边是一个小茶几，上面放着一杯热腾腾的茶和几块饼干，为这个朴素的卧室增添了一丝温馨和舒适。
"""


objects_in_text = prompt.invoke({"context": user_text})


In [14]:
print(objects_in_text)
# print(output)

['单人床', '木质床头柜', '台灯', '魔法书', '镜子', '书桌', '羽毛笔', '墨水瓶', '魔法课本', '书架', '魔法书籍', '药草学资料', '扶手椅', '小茶几', '茶', '饼干']


In [26]:
template = "{objects_in_text}中与{object_in_gellery}属于同一类物品的有哪些？输出一个列表"

prompt = ChatPromptTemplate.from_template(template) | llm | StrOutputParser()

object_in_gellery = ['床','双人床','上下铺','桌子','书桌','椅子','衣柜']

objects_matched = prompt.invoke({"objects_in_text": objects_in_text, "object_in_gellery": object_in_gellery})

In [27]:
print(objects_matched)

['单人床', '木质床头柜', '扶手椅', '小茶几']


In [17]:
# analyse_llm = ChatOllama(model="llama3:8b", temperature=0)

# from langchain import hub
# from langchain_core.prompts import PromptTemplate

# select_prompt = hub.pull("hwchase17/self-discovery-select")
# adapt_prompt = hub.pull("hwchase17/self-discovery-adapt")
# structured_prompt = hub.pull("hwchase17/self-discovery-structure")
# reasoning_prompt = hub.pull("hwchase17/self-discovery-reasoning")

# from langchain_core.runnables import RunnablePassthrough

# select_chain = select_prompt | analyse_llm | StrOutputParser()
# adapt_chain = adapt_prompt | analyse_llm | StrOutputParser()
# structure_chain = structured_prompt | analyse_llm | StrOutputParser()
# reasoning_chain = reasoning_prompt | analyse_llm | StrOutputParser()

# overall_chain = (
#     RunnablePassthrough.assign(selected_modules=select_chain)
#     .assign(adapted_modules=adapt_chain)
#     .assign(reasoning_structure=structure_chain)
#     .assign(answer=reasoning_chain)
# )

# reasoning_modules = [
#     "1. How could I devise an experiment to help solve that problem?",
#     "2. Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
#     "3. How could I measure progress on this problem?",
#     "4. How can I simplify the problem so that it is easier to solve?",
#     "5. What are the key assumptions underlying this problem?",
#     "6. What are the potential risks and drawbacks of each solution?",
#     "7. What are the alternative perspectives or viewpoints on this problem?",
#     "8. What are the long-term implications of this problem and its solutions?",
#     "9. How can I break down this problem into smaller, more manageable parts?",
#     "10. Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
#     "11. Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
#     "12. Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
#     "13. Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
#     "14. Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
#     "15. Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
#     "16. What is the core issue or problem that needs to be addressed?",
#     "17. What are the underlying causes or factors contributing to the problem?",
#     "18. Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
#     "19. What are the potential obstacles or challenges that might arise in solving this problem?",
#     "20. Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
#     "21. Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
#     "22. What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
#     "23. How can progress or success in solving the problem be measured or evaluated?",
#     "24. What indicators or metrics can be used?",
#     "25. Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
#     "26. Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
#     "27. Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
#     "28. Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
#     "29. Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
#     "30. Is the problem a design challenge that requires creative solutions and innovation?",
#     "31. Does the problem require addressing systemic or structural issues rather than just individual instances?",
#     "32. Is the problem time-sensitive or urgent, requiring immediate attention and action?",
#     "33. What kinds of solution typically are produced for this kind of problem specification?",
#     "34. Given the problem specification and the current best solution, have a guess about other possible solutions."
#     "35. Let’s imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?"
#     "36. What is the best way to modify this current best solution, given what you know about these kinds of problem specification?"
#     "37. Ignoring the current best solution, create an entirely new solution to the problem."
#     "38. Let’s think step by step."
#     "39. Let’s make a step by step plan and implement it with good notation and explanation.",
# ]

# reasoning_modules_str = "\n".join(reasoning_modules)

# output = overall_chain.invoke(
#     {"task_description": task, "reasoning_modules": reasoning_modules_str}
# )

# print(output['answer'])
